In [ ]:
#128 bins below 0
#128 bins above 0
#1 bin below -1
#1 bin above 1
bins = [i / 128 for i in range(-128, 129)]
print(bins)

In [ ]:
# bins.insert(0, float(-inf))
# bins.append(float(inf))
# print(bins)

In [3]:
bin_ranges = []
for i in range(len(bins) - 1):
    bin_ranges.append((bins[i], bins[i + 1]))

print(bin_ranges)

[(-1.0, -0.9921875), (-0.9921875, -0.984375), (-0.984375, -0.9765625), (-0.9765625, -0.96875), (-0.96875, -0.9609375), (-0.9609375, -0.953125), (-0.953125, -0.9453125), (-0.9453125, -0.9375), (-0.9375, -0.9296875), (-0.9296875, -0.921875), (-0.921875, -0.9140625), (-0.9140625, -0.90625), (-0.90625, -0.8984375), (-0.8984375, -0.890625), (-0.890625, -0.8828125), (-0.8828125, -0.875), (-0.875, -0.8671875), (-0.8671875, -0.859375), (-0.859375, -0.8515625), (-0.8515625, -0.84375), (-0.84375, -0.8359375), (-0.8359375, -0.828125), (-0.828125, -0.8203125), (-0.8203125, -0.8125), (-0.8125, -0.8046875), (-0.8046875, -0.796875), (-0.796875, -0.7890625), (-0.7890625, -0.78125), (-0.78125, -0.7734375), (-0.7734375, -0.765625), (-0.765625, -0.7578125), (-0.7578125, -0.75), (-0.75, -0.7421875), (-0.7421875, -0.734375), (-0.734375, -0.7265625), (-0.7265625, -0.71875), (-0.71875, -0.7109375), (-0.7109375, -0.703125), (-0.703125, -0.6953125), (-0.6953125, -0.6875), (-0.6875, -0.6796875), (-0.6796875, -0

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/expanse/lustre/projects/csd819/a3murali/cache/'

from huggingface_hub import login
login(token = "<hf_token>")

import transformers
import torch

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForCausalLM, pipeline, LlamaForCausalLM

model = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"
tokenizer = AutoTokenizer.from_pretrained(model)


llamatales_pipeline = pipeline("text-generation", model = model, device = "cuda")

# prompts = {
#     1: "Once upon a time there was a dragon", 
#     2: "Once upon a time there was a princess", 
#     3: "Once upon a time there were two children",
#     4: "Once upon a time there was a prince",
#     5: "Once upon a time there was a frog",
#     6: "Once upon a time there was a king",
#     7: "Once upon a time there was a queen",
#     8: "Once upon a time there was a wolf",
#     9: "Once upon a time there was a genie",
#     10: "Once upon a time there was a poor boy"
# }
prompts = {1: "Once upon a time there was a dragon"}
#generation - generate stories for each prompt
sequences = llamatales_pipeline(
    prompts[1],
    do_sample = True,
    top_k = 10,
    num_return_sequences = 1,
    max_new_tokens = 512
)
generated_story = sequences[0]['generated_text']
print(generated_story)

num_tokens_generated_story = len(tokenizer.encode(generated_story))

#test - collect hidden states

config = AutoConfig.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325", output_hidden_states = True)
model = AutoModelForCausalLM.from_config(config).to('cuda')

tokenizer = AutoTokenizer.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325/", config = config)

inputs = tokenizer(generated_story, return_tensors="pt").to("cuda")

outputs = model.generate(inputs.input_ids, attention_mask=inputs["attention_mask"], do_sample = True, top_k = 10, num_return_sequences = 1, max_new_tokens = 512, eos_token_id = tokenizer.eos_token_id, pad_token_id = tokenizer.pad_token_id, return_dict_in_generate=True)

print(outputs.hidden_states[:1])

convert_hidden_states = []

for i in outputs.hidden_states:
    convert_hidden_states.append([j.detach().cpu().numpy() for j in i])

print(len(convert_hidden_states))
print(len(convert_hidden_states[0]))
print(len(convert_hidden_states[0][0]))
print(len(convert_hidden_states[0][0][0]))
print(len(convert_hidden_states[0][0][0][0]))

print(len(convert_hidden_states) * len(convert_hidden_states[0]) * len(convert_hidden_states[0][0]) * len(convert_hidden_states[0][0][0]) * len(convert_hidden_states[0][0][0][0]))

convert_hidden_states = np.array([])

for i in outputs.hidden_states:
    for j in i:
        convert_hidden_states = np.append(convert_hidden_states, j.detach().cpu().numpy())

# print("Min Value: ", np.min(np.array(convert_hidden_states)))
# print("Max Value: ", np.max(convert_hidden_states))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: read).
Your token has been saved to /home/a3murali/.cache/huggingface/token
Login successful
PyTorch Version: 1.13.0a0+d321be6
Transformers Version: 4.35.0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Once upon a time there was a dragon. The dragon was very big and scary. He lived in a cave. One day, the dragon said, "I will be the strongest creature in the land." He was mean and breathed fire.

The dragon wanted the kingdom's treasure. The king said, "The dragon is not here to help us. He's not a friendly dragon." But the queen said, "No, I don't think that's a good idea. The kingdom needs a kind friend." So they went to the dragon to ask him how much he wanted to help them. The dragon said, "I can do a better job. I'll give you a special task."

The kingdom was sad and said, "We need someone to help us." But the dragon just laughed and said, "I don't need to help. I will do it myself." The dragon used his fire to light a big fire and made the kingdom go dark. Then, the kingdom started to disappear. The people of the kingdom were trapped, and the dragon was left alone in the dark. The people said, "We miss our kind friend, the king."

The king and queen were sad. They wanted their 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (512). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


((tensor([[[-0.0113, -0.0261,  0.0201,  ...,  0.0018,  0.0341, -0.0012],
         [ 0.0033, -0.0344,  0.0053,  ..., -0.0281,  0.0047, -0.0484],
         [-0.0171,  0.0029,  0.0054,  ..., -0.0119, -0.0021, -0.0067],
         ...,
         [-0.0328,  0.0076,  0.0059,  ...,  0.0085,  0.0026, -0.0034],
         [-0.0136, -0.0042, -0.0017,  ...,  0.0433,  0.0067,  0.0043],
         [-0.0075, -0.0111, -0.0146,  ..., -0.0316,  0.0212, -0.0185]]],
       device='cuda:0'), tensor([[[ 0.2622, -0.0271, -0.1572,  ..., -0.4707,  0.6736, -0.4506],
         [ 0.2784, -0.0843, -0.0334,  ..., -0.4175,  0.5123, -0.3506],
         [ 0.2110,  0.0941, -0.0591,  ..., -0.1643,  0.2500, -0.2256],
         ...,
         [ 0.1558,  0.1417,  0.0469,  ...,  0.2122,  0.0154, -0.1033],
         [-0.0196,  0.0330, -0.0048,  ...,  0.1516,  0.0030,  0.0158],
         [-0.0184,  0.0482, -0.0476,  ...,  0.1827,  0.0080, -0.1848]]],
       device='cuda:0'), tensor([[[ 0.1318, -0.3516,  0.3153,  ..., -0.4985,  0.7155, -0.

In [5]:
print(type(convert_hidden_states))
print(convert_hidden_states.shape)
hidden_vals = convert_hidden_states.flatten()
print(hidden_vals.shape)
bin_counts = [0 for i in range(len(bin_ranges) + 2)]
# bin_counts.append(0)
# bin_counts.append(0)
print(hidden_vals)
for i in hidden_vals:
    for j in range(len(bin_ranges)):
        if(i >= bin_ranges[j][0] and i < bin_ranges[j][1]):
            bin_counts[j + 1] += 1
    if(i < -1):
        bin_counts[0] += 1
    elif(i >= 1):
        bin_counts[len(bin_counts) - 1] += 1


# plt.hist(bin_counts, bins = 258)
# plt.show()
print(bin_counts)

<class 'numpy.ndarray'>
(3898368,)
(3898368,)
[-0.01130614 -0.0261382   0.02006644 ...  0.77043104  1.76886702
 -0.55354446]


KeyboardInterrupt: 